# Qube - Data Challenge

### Raphaël Barthès

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

In [2]:
test_away_player = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Test_Data/test_away_player_statistics_df.csv")
test_away_team = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Test_Data/test_away_team_statistics_df.csv")
test_home_player = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Test_Data/test_home_player_statistics_df.csv")
test_home_team = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Test_Data/test_home_team_statistics_df.csv")

In [3]:
train_away_player = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Train_Data/train_away_player_statistics_df.csv")
train_away_team = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Train_Data/train_away_team_statistics_df.csv")
train_home_player = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Train_Data/train_home_player_statistics_df.csv")
train_home_team = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Train_Data/train_home_team_statistics_df.csv")
y = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Y_train_1rknArQ.csv")

In [4]:
train_away_player.head()

,ID,LEAGUE,TEAM_NAME,POSITION,PLAYER_NAME,PLAYER_ACCURATE_CROSSES_season_sum,PLAYER_ACCURATE_PASSES_season_sum,PLAYER_AERIALS_WON_season_sum,PLAYER_ASSISTS_season_sum,PLAYER_BIG_CHANCES_CREATED_season_sum,...,PLAYER_STARTING_LINEUP_5_last_match_std,PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_std,PLAYER_TACKLES_5_last_match_std,PLAYER_TOTAL_CROSSES_5_last_match_std,PLAYER_TOTAL_DUELS_5_last_match_std,PLAYER_YELLOWCARDS_5_last_match_std,PLAYER_PUNCHES_5_last_match_std,PLAYER_LONG_BALLS_5_last_match_std,PLAYER_LONG_BALLS_WON_5_last_match_std,PLAYER_SHOTS_OFF_TARGET_5_last_match_std
0,0,Ligue 1,Olympique Marseille,defender,Lucas Perrin,0.0,19.0,20.0,0.0,0.0,...,70.0,30.0,46.0,0.0,79.0,99.0,NaN,NaN,NaN,NaN
1,0,Ligue 1,Olympique Marseille,midfielder,Kevin Strootman,2.0,48.0,19.0,28.0,8.0,...,77.0,0.0,40.0,6.0,46.0,77.0,NaN,NaN,NaN,NaN
2,0,Ligue 1,Olympique Marseille,goalkeeper,Yohann Pelé,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,0,Ligue 1,Olympique Marseille,defender,Abdallah Ali Mohamed,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Ligue 1,Olympique Marseille,NaN,Nemanja Radonjic,8.0,11.0,4.0,0.0,0.0,...,77.0,69.0,21.0,13.0,66.0,0.0,NaN,NaN,NaN,NaN


In [5]:
def encode_outcome(row):
    if row['HOME_WINS'] == 1:
        return 0
    elif row['DRAW'] == 1:
        return 1
    elif row['AWAY_WINS'] == 1:
        return 2

y['OUTCOME'] = y.apply(encode_outcome, axis=1)
y

,ID,HOME_WINS,DRAW,AWAY_WINS,OUTCOME
0,0,0,0,1,2
1,1,0,1,0,1
2,2,0,0,1,2
3,3,1,0,0,0
4,4,0,1,0,1
...,...,...,...,...,...
12298,12298,0,0,1,2
12299,12299,0,0,1,2
12300,12300,0,0,1,2
12301,12301,1,0,0,0


## Test Benchmarks

In [11]:
columns_to_drop = ['ID', 'LEAGUE', 'TEAM_NAME']
train_home_team_bis = train_home_team.drop(columns_to_drop, axis=1)
train_away_team_bis = train_away_team.drop(columns_to_drop, axis=1)

# Add 'away' suffix to column names of train_away_team
train_away_team_renamed = train_away_team_bis.add_suffix('_away')

# Add 'home' suffix to column names of train_home_team
train_home_team_renamed = train_home_team_bis.add_suffix('_home')

X = pd.concat([train_home_team_renamed, train_away_team_renamed], axis=1)

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

y_outcome = y['OUTCOME']

X_train, X_test, y_train, y_test = train_test_split(X, y_outcome, test_size=0.1, random_state=42)

# XGBoost classifier
xg_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, colsample_bytree=0.3, learning_rate=0.1,
                            max_depth=5, alpha=10, n_estimators=10)

xg_clf.fit(X_train, y_train)
y_pred = xg_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5158407798537774


In [13]:
y_pred = np.zeros(len(y_test))
# Accuracy of model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4565393988627132


## Preparing player Datasets

In [9]:
## Note: By doing the following, we lose the IDs (matchs) where no position has been specified for any player
## We lose approx 1000 IDs out of 12500

columns_to_drop = ['LEAGUE', 'TEAM_NAME', 'PLAYER_NAME']
train_away_player_avg = train_away_player.drop(columns_to_drop, axis=1)

# Get all unique IDs and positions
unique_ids = train_away_player_avg['ID'].unique()
train_away_player_avg = train_away_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

unique_positions = sorted(train_away_player_avg['POSITION'].dropna().unique())

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Assuming all_combinations_df and train_away_player_avg are your DataFrames
train_away_player_avg_full = pd.merge(all_combinations_df, train_away_player_avg, on=['ID', 'POSITION'], how='left', indicator=True)
train_away_player_avg_full = train_away_player_avg_full.drop('_merge', axis=1)

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_away = train_away_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_away.columns = ['_'.join(col) for col in df_player_away.columns.values]
df_player_away.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_away= pd.merge(all_ids_df, df_player_away, on='ID', how='left')

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_away= pd.merge(all_ids_df, df_player_away, on='ID', how='left')

# Add 'home' suffix to column names
df_player_away = df_player_away.add_suffix('_away')
df_player_away.rename(columns={'ID_away': 'ID'}, inplace=True)

In [10]:
## Note: By doing the following, we lose the IDs (matchs) where no position has been specified for any player
## We lose approx 1000 IDs out of 12500

columns_to_drop = ['LEAGUE', 'TEAM_NAME', 'PLAYER_NAME']
train_home_player_avg = train_home_player.drop(columns_to_drop, axis=1)
# Get all unique IDs and positions
unique_ids = train_home_player_avg['ID'].unique()
unique_positions = sorted(train_home_player_avg['POSITION'].dropna().unique())

train_home_player_avg = train_home_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Merge with the average dataset
train_home_player_avg_full = pd.merge(all_combinations_df, train_home_player_avg, on=['ID', 'POSITION'], how='left')

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_home = train_home_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_home.columns = ['_'.join(col) for col in df_player_home.columns.values]
df_player_home.reset_index(inplace=True)

all_ids_df = pd.DataFrame({'ID': unique_ids})

df_player_home= pd.merge(all_ids_df, df_player_home, on='ID', how='left')

# Add 'home' suffix to column names
df_player_home = df_player_home.add_suffix('_home')
df_player_home.rename(columns={'ID_home': 'ID'}, inplace=True)

In [11]:
df_player = pd.merge(df_player_home, df_player_away, on='ID')

# Adding player dataset to team 

In [12]:
columns_to_drop = ['ID', 'LEAGUE', 'TEAM_NAME']
train_home_team_bis = train_home_team.drop(columns_to_drop, axis=1)
train_away_team_bis = train_away_team.drop(columns_to_drop, axis=1)

# Add 'away' suffix to column names of train_away_team
train_away_team_renamed = train_away_team_bis.add_suffix('_away')

# Add 'home' suffix to column names of train_home_team
train_home_team_renamed = train_home_team_bis.add_suffix('_home')

train = pd.concat([train_home_team_renamed, train_away_team_renamed], axis=1)
train['ID'] = range(len(train))
train

,TEAM_SHOTS_TOTAL_season_sum_home,TEAM_SHOTS_INSIDEBOX_season_sum_home,TEAM_SHOTS_OFF_TARGET_season_sum_home,TEAM_SHOTS_ON_TARGET_season_sum_home,TEAM_SHOTS_OUTSIDEBOX_season_sum_home,TEAM_PASSES_season_sum_home,TEAM_SUCCESSFUL_PASSES_season_sum_home,TEAM_SAVES_season_sum_home,TEAM_CORNERS_season_sum_home,TEAM_FOULS_season_sum_home,...,TEAM_REDCARDS_5_last_match_std_away,TEAM_OFFSIDES_5_last_match_std_away,TEAM_ATTACKS_5_last_match_std_away,TEAM_PENALTIES_5_last_match_std_away,TEAM_SUBSTITUTIONS_5_last_match_std_away,TEAM_BALL_SAFE_5_last_match_std_away,TEAM_DANGEROUS_ATTACKS_5_last_match_std_away,TEAM_INJURIES_5_last_match_std_away,TEAM_GOALS_5_last_match_std_away,ID
0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,3.0,6.0,...,5.0,4.0,0.0,6.0,8.0,4.0,3.0,2.0,3.0,0
1,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,6.0,8.0,...,0.0,3.0,1.0,8.0,4.0,10.0,0.0,5.0,3.0,1
2,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,2.0,7.0,...,10.0,4.0,4.0,0.0,8.0,3.0,0.0,9.0,6.0,2
3,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,2.0,0.0,...,0.0,1.0,2.0,0.0,5.0,6.0,3.0,NaN,2.0,3
4,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,...,0.0,4.0,4.0,9.0,4.0,1.0,4.0,6.0,5.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12298,4.0,2.0,3.0,4.0,7.0,4.0,4.0,3.0,2.0,2.0,...,5.0,0.0,1.0,6.0,10.0,1.0,1.0,NaN,2.0,12298
12299,4.0,2.0,3.0,1.0,5.0,1.0,1.0,9.0,1.0,10.0,...,5.0,2.0,2.0,0.0,0.0,1.0,6.0,9.0,1.0,12299
12300,4.0,3.0,5.0,3.0,5.0,1.0,1.0,6.0,1.0,8.0,...,0.0,3.0,2.0,0.0,0.0,6.0,3.0,4.0,4.0,12300
12301,2.0,NaN,1.0,1.0,NaN,NaN,NaN,0.0,4.0,2.0,...,0.0,NaN,0.0,8.0,3.0,1.0,2.0,NaN,6.0,12301


In [13]:
train = pd.merge(train, df_player, on='ID')

In [19]:
def encode_outcome(row):
    if row['HOME_WINS'] == 1:
        return 0
    elif row['DRAW'] == 1:
        return 1
    elif row['AWAY_WINS'] == 1:
        return 2

y['OUTCOME'] = y.apply(encode_outcome, axis=1)
y_bis = y[['ID', 'OUTCOME']]

In [20]:
train = pd.merge(train, y_bis, on='ID')

In [ ]:
train

## Run PCA

In [57]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

y_outcome = train['OUTCOME']
columns_to_drop = ['ID', 'OUTCOME']
X = train.drop(columns_to_drop, axis=1)
imputer = SimpleImputer(strategy='mean') 
data_imputed = imputer.fit_transform(X)

# Step 2: Scaling/Normalization
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_imputed)

# Step 3: PCA
# Choose the number of components based on your requirements or using explained variance ratio
n_components = 700
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(scaled_data)

# Explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
total_variance_explained = sum(explained_variance_ratio)

#print("Explained Variance Ratio:", explained_variance_ratio)
print("Total Variance Explained by", n_components, "components:", total_variance_explained)

Total Variance Explained by 700 components: 0.9463266103221084


## Model with PCA

In [58]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(pca_result, y_outcome, test_size=0.1, random_state=42)

# XGBoost classifier
xg_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, colsample_bytree=0.3, learning_rate=0.1,
                            max_depth=5, alpha=150, n_estimators=10)

xg_clf.fit(X_train, y_train)
y_pred = xg_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.49634443541835904


In [63]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

# XGBoost classifier
xg_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, colsample_bytree=0.3, learning_rate=0.1,
                            max_depth=5, alpha=10, n_estimators=10)

# Cross-validation strategy
cv = StratifiedKFold(n_splits=8, shuffle=True, random_state=42)
cv_scores = cross_val_score(xg_clf, pca_result, y_outcome, cv=cv, scoring='accuracy')

# Cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())

Cross-Validation Scores: [0.45968791 0.46618986 0.47919376 0.47204161 0.4603381  0.46488947
 0.45643693 0.4710475 ]
Mean Accuracy: 0.46622814105129395


## Use on test set

In [28]:
# Add 'away' suffix to column names of train_away_team
test_away_team_renamed = test_away_team.add_suffix('_away')
test_away_team_renamed.rename(columns={'ID_away': 'ID'}, inplace=True)

# Add 'home' suffix to column names of train_home_team
test_home_team_renamed = test_home_team.add_suffix('_home')
test_home_team_renamed.rename(columns={'ID_home': 'ID'}, inplace=True)

In [29]:
test = pd.merge(test_home_team_renamed, test_away_team_renamed, on='ID')

### Player dataset

In [30]:
test_away_player_avg = test_away_player
# Get all unique IDs and positions
unique_ids = test_away_player_avg['ID'].unique()
unique_positions = sorted(test_away_player_avg['POSITION'].dropna().unique())

test_away_player_avg = test_away_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()
print(len(all_combinations_df))

# Merge with the average dataset
test_away_player_avg_full = pd.merge(all_combinations_df, test_away_player_avg, on=['ID', 'POSITION'], how='left')

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_away = test_away_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_away.columns = ['_'.join(col) for col in df_player_away.columns.values]
df_player_away.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_away= pd.merge(all_ids_df, df_player_away, on='ID', how='left')

# Add 'home' suffix to column names
df_player_away = df_player_away.add_suffix('_away')
df_player_away.rename(columns={'ID_away': 'ID'}, inplace=True)

101472


In [31]:
test_home_player_avg = test_home_player

# Get all unique IDs and positions
unique_ids = test_home_player_avg['ID'].unique()
unique_positions = sorted(test_home_player_avg['POSITION'].dropna().unique())

test_home_player_avg = test_home_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Merge with the average dataset
test_home_player_avg_full = pd.merge(all_combinations_df, test_home_player_avg, on=['ID', 'POSITION'], how='left')

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_home = test_home_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_home.columns = ['_'.join(col) for col in df_player_home.columns.values]
df_player_home.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_home= pd.merge(all_ids_df, df_player_home, on='ID', how='left')

# Add 'home' suffix to column names
df_player_home = df_player_home.add_suffix('_home')
df_player_home.rename(columns={'ID_home': 'ID'}, inplace=True)

In [32]:
df_player = pd.merge(df_player_home, df_player_away, on='ID')

In [33]:
test = pd.merge(test, df_player, on='ID')

In [40]:
columns_to_drop = ['ID']
X_TEST = test.drop(columns_to_drop, axis=1)

# Assuming X_test and X_TEST are pandas DataFrames
# Get the columns of each DataFrame
columns_X_train = set(train.columns)
columns_X_TEST = set(X_TEST.columns)

# Find columns that are in one DataFrame but not in the other
columns_missing_in_X_train = columns_X_TEST - columns_X_train
X_TEST = X_TEST.drop(columns_missing_in_X_train, axis=1)

In [ ]:
# Step 1: Impute missing values in X_TEST
X_TEST_imputed = imputer.transform(X_TEST)

# Step 2: Scale the imputed X_TEST data
X_TEST_scaled = scaler.transform(X_TEST_imputed)

# Step 3: Apply PCA transformation to X_TEST
X_TEST_pca = pca.transform(X_TEST_scaled)

# Step 4: Predict outcomes using the trained XGBoost classifier
y_pred = xg_clf.predict(X_TEST_pca)


In [39]:
y_pred = xg_clf.predict(X_TEST)
y_pred

array([0, 2, 0, ..., 0, 0, 0], dtype=int32)

In [42]:
(y_pred == 0).sum()/len(y_pred)

0.723076316619363

In [43]:
test['OUTCOME'] = y_pred
Y_test = test[['ID', 'OUTCOME']]
Y_test['HOME_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 0 else 0)
Y_test['DRAW'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 1 else 0)
Y_test['AWAY_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 2 else 0)
Y_test = Y_test[['ID' , 'HOME_WINS' , 'DRAW' , 'AWAY_WINS']]

/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_55859/4064822606.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['HOME_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 0 else 0)
/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_55859/4064822606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['DRAW'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 1 else 0)
/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_55859/4064822606.py:5: SettingWithCopyWarning: 
A 

In [47]:
Y_test

,ID,HOME_WINS,DRAW,AWAY_WINS
0,12303,1,0,0
1,12304,0,0,1
2,12305,1,0,0
3,12306,1,0,0
4,12307,0,0,1
...,...,...,...,...
25363,37666,0,0,1
25364,37667,1,0,0
25365,37668,1,0,0
25366,37669,1,0,0


In [48]:
exemple = pd.read_csv("/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Y_test_random_sEE2QeA.csv")
exemple

,ID,HOME_WINS,DRAW,AWAY_WINS
0,12303,0,1,0
1,12304,0,1,0
2,12305,0,1,0
3,12306,0,1,0
4,12307,1,0,0
...,...,...,...,...
25363,37666,0,1,0
25364,37667,0,1,0
25365,37668,0,0,1
25366,37669,1,0,0


In [50]:
Y_test.set_index('ID', inplace=True)
Y_test.to_csv('/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Y_test.csv')

## Simpler model - Best results so far 

In [79]:
columns_to_keep = ['TEAM_SHOTS_TOTAL_season_average',
                   'TEAM_SHOTS_INSIDEBOX_season_average',
                   'TEAM_SHOTS_OFF_TARGET_season_average',
                   'TEAM_SHOTS_ON_TARGET_season_average',
                   'TEAM_SHOTS_OUTSIDEBOX_season_average',
                   'TEAM_PASSES_season_average',
                   'TEAM_SUCCESSFUL_PASSES_season_average',
                   'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average',
                   'TEAM_SAVES_season_average', 'TEAM_CORNERS_season_average',
                   'TEAM_BALL_POSSESSION_season_average', 'TEAM_FOULS_season_average',
                   'TEAM_YELLOWCARDS_season_average', 'TEAM_REDCARDS_season_average',
                   'TEAM_OFFSIDES_season_average', 'TEAM_ATTACKS_season_average',
                   'TEAM_PENALTIES_season_average',
                   'TEAM_SUBSTITUTIONS_season_average',
                   'TEAM_BALL_SAFE_season_average',
                   'TEAM_DANGEROUS_ATTACKS_season_average',
                   'TEAM_INJURIES_season_average', 'TEAM_GOALS_season_average',
                   'TEAM_GAME_WON_season_average', 'TEAM_GAME_DRAW_season_average',
                   'TEAM_GAME_LOST_season_average',
                   'TEAM_SHOTS_TOTAL_5_last_match_average',
                   'TEAM_SHOTS_INSIDEBOX_5_last_match_average',
                   'TEAM_SHOTS_OFF_TARGET_5_last_match_average',
                   'TEAM_SHOTS_ON_TARGET_5_last_match_average',
                   'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average',
                   'TEAM_PASSES_5_last_match_average',
                   'TEAM_SUCCESSFUL_PASSES_5_last_match_average',
                   'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average',
                   'TEAM_SAVES_5_last_match_average',
                   'TEAM_CORNERS_5_last_match_average',
                   'TEAM_BALL_POSSESSION_5_last_match_average',
                   'TEAM_FOULS_5_last_match_average',
                   'TEAM_YELLOWCARDS_5_last_match_average',
                   'TEAM_REDCARDS_5_last_match_average',
                   'TEAM_OFFSIDES_5_last_match_average',
                   'TEAM_ATTACKS_5_last_match_average',
                   'TEAM_PENALTIES_5_last_match_average',
                   'TEAM_SUBSTITUTIONS_5_last_match_average',
                   'TEAM_BALL_SAFE_5_last_match_average',
                   'TEAM_DANGEROUS_ATTACKS_5_last_match_average',
                   'TEAM_INJURIES_5_last_match_average',
                   'TEAM_GOALS_5_last_match_average',
                   'TEAM_GAME_WON_5_last_match_average',
                   'TEAM_GAME_DRAW_5_last_match_average',
                   'TEAM_GAME_LOST_5_last_match_average']

In [80]:
train_home_team_bis = train_home_team[columns_to_keep]
train_away_team_bis = train_away_team[columns_to_keep]

# Add 'away' suffix to column names of train_away_team
train_away_team_renamed = train_away_team_bis.add_suffix('_away')

# Add 'home' suffix to column names of train_home_team
train_home_team_renamed = train_home_team_bis.add_suffix('_home')


In [81]:
X = pd.concat([train_home_team_renamed, train_away_team_renamed], axis=1)

In [89]:
## Best so far
from sklearn.model_selection import cross_val_score

# Define the XGBoost classifier
xg_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, colsample_bytree=0.3, learning_rate=0.01,
                            max_depth=10, alpha=10, n_estimators=100)

# Perform K-fold cross-validation
scores = cross_val_score(xg_clf, X, y_outcome, cv=10)  # Change cv to the desired number of folds

# Print the accuracy scores for each fold
print("Accuracy scores for each fold:", scores)

# Compute the mean accuracy across all folds
mean_accuracy = scores.mean()
print("Mean accuracy:", mean_accuracy)

Accuracy scores for each fold: [0.48822096 0.48009748 0.47684809 0.49268293 0.47886179 0.48943089
 0.50243902 0.50243902 0.51138211 0.48617886]
Mean accuracy: 0.49085811654217276


In [61]:
xg_clf.fit(X_standardized, y_outcome)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3, ...)

In [62]:
columns_to_keep += ['ID']

test_home_team_bis = test_home_team[columns_to_keep]
test_away_team_bis = test_away_team[columns_to_keep]

test_away_team_renamed = test_away_team.add_suffix('_away')
test_away_team_renamed.rename(columns={'ID_away': 'ID'}, inplace=True)

# Add 'home' suffix to column names of train_home_team
test_home_team_renamed = test_home_team.add_suffix('_home')
test_home_team_renamed.rename(columns={'ID_home': 'ID'}, inplace=True)
test = pd.merge(test_home_team_renamed, test_away_team_renamed, on='ID')

In [68]:
columns_to_drop = ['ID']
X_TEST = test.drop(columns_to_drop, axis=1)

# Assuming X_test and X_TEST are pandas DataFrames
# Get the columns of each DataFrame
columns_X = set(X.columns)
columns_X_TEST = set(X_TEST.columns)

# Find columns that are in one DataFrame but not in the other
columns_missing_in_X = columns_X_TEST - columns_X
X_TEST = X_TEST.drop(columns_missing_in_X, axis=1)

In [70]:
len(X_TEST.columns)

100

In [64]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Initialize SimpleImputer with mean strategy
imputer = SimpleImputer(strategy='mean')

# Impute missing values in X
X_imputed = imputer.fit_transform(X_TEST)

# Initialize StandardScaler
scaler = StandardScaler()

# Standardize the features in X
X_TEST = scaler.fit_transform(X_imputed)

In [65]:
y_pred = xg_clf.predict(X_TEST)
len(y_pred)

ValueError: Feature shape mismatch, expected: 102, got 100

In [52]:
test['OUTCOME'] = y_pred
Y_test = test[['ID', 'OUTCOME']]
Y_test['HOME_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 0 else 0)
Y_test['DRAW'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 1 else 0)
Y_test['AWAY_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 2 else 0)
Y_test = Y_test[['ID' , 'HOME_WINS' , 'DRAW' , 'AWAY_WINS']]

/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_73122/4064822606.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['HOME_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 0 else 0)
/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_73122/4064822606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['DRAW'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 1 else 0)
/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_73122/4064822606.py:5: SettingWithCopyWarning: 
A 

In [53]:
Y_test.set_index('ID', inplace=True)
Y_test

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
12303,1,0,0
12304,0,0,1
12305,1,0,0
12306,1,0,0
12307,0,0,1
...,...,...,...
37666,0,0,1
37667,1,0,0
37668,1,0,0


In [54]:
Y_test.to_csv('/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Y_test.csv')

## With player dataset but removing useless columns 

In [16]:
columns_to_keep = ['TEAM_SHOTS_TOTAL_season_average',
                   'TEAM_SHOTS_INSIDEBOX_season_average',
                   'TEAM_SHOTS_OFF_TARGET_season_average',
                   'TEAM_SHOTS_ON_TARGET_season_average',
                   'TEAM_SHOTS_OUTSIDEBOX_season_average',
                   'TEAM_PASSES_season_average',
                   'TEAM_SUCCESSFUL_PASSES_season_average',
                   'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average',
                   'TEAM_SAVES_season_average', 'TEAM_CORNERS_season_average',
                   'TEAM_BALL_POSSESSION_season_average', 'TEAM_FOULS_season_average',
                   'TEAM_YELLOWCARDS_season_average', 'TEAM_REDCARDS_season_average',
                   'TEAM_OFFSIDES_season_average', 'TEAM_ATTACKS_season_average',
                   'TEAM_PENALTIES_season_average',
                   'TEAM_SUBSTITUTIONS_season_average',
                   'TEAM_BALL_SAFE_season_average',
                   'TEAM_DANGEROUS_ATTACKS_season_average',
                   'TEAM_INJURIES_season_average', 'TEAM_GOALS_season_average',
                   'TEAM_GAME_WON_season_average', 'TEAM_GAME_DRAW_season_average',
                   'TEAM_GAME_LOST_season_average',
                   'TEAM_SHOTS_TOTAL_5_last_match_average',
                   'TEAM_SHOTS_INSIDEBOX_5_last_match_average',
                   'TEAM_SHOTS_OFF_TARGET_5_last_match_average',
                   'TEAM_SHOTS_ON_TARGET_5_last_match_average',
                   'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average',
                   'TEAM_PASSES_5_last_match_average',
                   'TEAM_SUCCESSFUL_PASSES_5_last_match_average',
                   'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average',
                   'TEAM_SAVES_5_last_match_average',
                   'TEAM_CORNERS_5_last_match_average',
                   'TEAM_BALL_POSSESSION_5_last_match_average',
                   'TEAM_FOULS_5_last_match_average',
                   'TEAM_YELLOWCARDS_5_last_match_average',
                   'TEAM_REDCARDS_5_last_match_average',
                   'TEAM_OFFSIDES_5_last_match_average',
                   'TEAM_ATTACKS_5_last_match_average',
                   'TEAM_PENALTIES_5_last_match_average',
                   'TEAM_SUBSTITUTIONS_5_last_match_average',
                   'TEAM_BALL_SAFE_5_last_match_average',
                   'TEAM_DANGEROUS_ATTACKS_5_last_match_average',
                   'TEAM_INJURIES_5_last_match_average',
                   'TEAM_GOALS_5_last_match_average',
                   'TEAM_GAME_WON_5_last_match_average',
                   'TEAM_GAME_DRAW_5_last_match_average',
                   'TEAM_GAME_LOST_5_last_match_average']

In [17]:
train_home_team_bis = train_home_team[columns_to_keep]
train_away_team_bis = train_away_team[columns_to_keep]

# Add 'away' suffix to column names of train_away_team
train_away_team_renamed = train_away_team_bis.add_suffix('_away')

# Add 'home' suffix to column names of train_home_team
train_home_team_renamed = train_home_team_bis.add_suffix('_home')


In [18]:
columns_to_keep = ['ID', 'POSITION',
                   'PLAYER_ACCURATE_CROSSES_5_last_match_average',
                   'PLAYER_ACCURATE_PASSES_5_last_match_average',
                   'PLAYER_ACCURATE_PASSES_PERCENTAGE_5_last_match_average',
                   'PLAYER_AERIALS_WON_5_last_match_average',
                   'PLAYER_ASSISTS_5_last_match_average',
                   'PLAYER_BIG_CHANCES_CREATED_5_last_match_average',
                   'PLAYER_BIG_CHANCES_MISSED_5_last_match_average',
                   'PLAYER_BLOCKED_SHOTS_5_last_match_average',
                   'PLAYER_CAPTAIN_5_last_match_average',
                   'PLAYER_CLEARANCES_5_last_match_average',
                   'PLAYER_CLEARANCE_OFFLINE_5_last_match_average',
                   'PLAYER_DISPOSSESSED_5_last_match_average',
                   'PLAYER_DRIBBLED_ATTEMPTS_5_last_match_average',
                   'PLAYER_DRIBBLED_PAST_5_last_match_average',
                   'PLAYER_DUELS_LOST_5_last_match_average',
                   'PLAYER_DUELS_WON_5_last_match_average',
                   'PLAYER_ERROR_LEAD_TO_GOAL_5_last_match_average',
                   'PLAYER_FOULS_5_last_match_average',
                   'PLAYER_FOULS_DRAWN_5_last_match_average',
                   'PLAYER_GOALKEEPER_GOALS_CONCEDED_5_last_match_average',
                   'PLAYER_GOALS_5_last_match_average',
                   'PLAYER_GOALS_CONCEDED_5_last_match_average',
                   'PLAYER_HIT_WOODWORK_5_last_match_average',
                   'PLAYER_INTERCEPTIONS_5_last_match_average',
                   'PLAYER_KEY_PASSES_5_last_match_average',
                   'PLAYER_MINUTES_PLAYED_5_last_match_average',
                   'PLAYER_OFFSIDES_5_last_match_average',
                   'PLAYER_OWN_GOALS_5_last_match_average',
                   'PLAYER_PASSES_5_last_match_average',
                   'PLAYER_PENALTIES_COMMITTED_5_last_match_average',
                   'PLAYER_PENALTIES_MISSES_5_last_match_average',
                   'PLAYER_PENALTIES_SAVED_5_last_match_average',
                   'PLAYER_PENALTIES_SCORED_5_last_match_average',
                   'PLAYER_PENALTIES_WON_5_last_match_average',
                   'PLAYER_RATING_5_last_match_average',
                   'PLAYER_REDCARDS_5_last_match_average',
                   'PLAYER_SAVES_5_last_match_average',
                   'PLAYER_SAVES_INSIDE_BOX_5_last_match_average',
                   'PLAYER_SHOTS_BLOCKED_5_last_match_average',
                   'PLAYER_SHOTS_ON_TARGET_5_last_match_average',
                   'PLAYER_SHOTS_TOTAL_5_last_match_average',
                   'PLAYER_STARTING_LINEUP_5_last_match_average',
                   'PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_average',
                   'PLAYER_TACKLES_5_last_match_average',
                   'PLAYER_TOTAL_CROSSES_5_last_match_average',
                   'PLAYER_TOTAL_DUELS_5_last_match_average',
                   'PLAYER_YELLOWCARDS_5_last_match_average',
                   'PLAYER_PUNCHES_5_last_match_average',
                   'PLAYER_LONG_BALLS_5_last_match_average',
                   'PLAYER_LONG_BALLS_WON_5_last_match_average',
                   'PLAYER_SHOTS_OFF_TARGET_5_last_match_average', 
                  'PLAYER_ACCURATE_CROSSES_season_average',
       'PLAYER_ACCURATE_PASSES_season_average',
       'PLAYER_ACCURATE_PASSES_PERCENTAGE_season_average',
       'PLAYER_AERIALS_WON_season_average',
       'PLAYER_ASSISTS_season_average',
       'PLAYER_BIG_CHANCES_CREATED_season_average',
       'PLAYER_BIG_CHANCES_MISSED_season_average',
       'PLAYER_BLOCKED_SHOTS_season_average',
       'PLAYER_CAPTAIN_season_average',
       'PLAYER_CLEARANCES_season_average',
       'PLAYER_CLEARANCE_OFFLINE_season_average',
       'PLAYER_DISPOSSESSED_season_average',
       'PLAYER_DRIBBLED_ATTEMPTS_season_average',
       'PLAYER_DRIBBLED_PAST_season_average',
       'PLAYER_DUELS_LOST_season_average',
       'PLAYER_DUELS_WON_season_average',
       'PLAYER_ERROR_LEAD_TO_GOAL_season_average',
       'PLAYER_FOULS_season_average', 'PLAYER_FOULS_DRAWN_season_average',
       'PLAYER_GOALKEEPER_GOALS_CONCEDED_season_average',
       'PLAYER_GOALS_season_average',
       'PLAYER_GOALS_CONCEDED_season_average',
       'PLAYER_HIT_WOODWORK_season_average',
       'PLAYER_INTERCEPTIONS_season_average',
       'PLAYER_KEY_PASSES_season_average',
       'PLAYER_MINUTES_PLAYED_season_average',
       'PLAYER_OFFSIDES_season_average',
       'PLAYER_OWN_GOALS_season_average', 'PLAYER_PASSES_season_average',
       'PLAYER_PENALTIES_COMMITTED_season_average',
       'PLAYER_PENALTIES_MISSES_season_average',
       'PLAYER_PENALTIES_SAVED_season_average',
       'PLAYER_PENALTIES_SCORED_season_average',
       'PLAYER_PENALTIES_WON_season_average',
       'PLAYER_RATING_season_average', 'PLAYER_REDCARDS_season_average',
       'PLAYER_SAVES_season_average',
       'PLAYER_SAVES_INSIDE_BOX_season_average',
       'PLAYER_SHOTS_BLOCKED_season_average',
       'PLAYER_SHOTS_ON_TARGET_season_average',
       'PLAYER_SHOTS_TOTAL_season_average',
       'PLAYER_STARTING_LINEUP_season_average',
       'PLAYER_SUCCESSFUL_DRIBBLES_season_average',
       'PLAYER_TACKLES_season_average',
       'PLAYER_TOTAL_CROSSES_season_average',
       'PLAYER_TOTAL_DUELS_season_average',
       'PLAYER_YELLOWCARDS_season_average',
       'PLAYER_PUNCHES_season_average',
       'PLAYER_LONG_BALLS_season_average',
       'PLAYER_LONG_BALLS_WON_season_average',
       'PLAYER_SHOTS_OFF_TARGET_season_average']

train_home_player_avg = train_home_player[columns_to_keep]
train_away_player_avg = train_away_player[columns_to_keep]

In [19]:
# Get all unique IDs and positions
unique_ids = train_away_player_avg['ID'].unique()
train_away_player_avg = train_away_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

unique_positions = sorted(train_away_player_avg['POSITION'].dropna().unique())

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Assuming all_combinations_df and train_away_player_avg are your DataFrames
train_away_player_avg_full = pd.merge(all_combinations_df, train_away_player_avg, on=['ID', 'POSITION'], how='left', indicator=True)
train_away_player_avg_full = train_away_player_avg_full.drop('_merge', axis=1)

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_away = train_away_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_away.columns = ['_'.join(col) for col in df_player_away.columns.values]
df_player_away.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_away= pd.merge(all_ids_df, df_player_away, on='ID', how='left')

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_away= pd.merge(all_ids_df, df_player_away, on='ID', how='left')

# Add 'home' suffix to column names
df_player_away = df_player_away.add_suffix('_away')
df_player_away.rename(columns={'ID_away': 'ID'}, inplace=True)

In [20]:
# Get all unique IDs and positions
unique_ids = train_home_player_avg['ID'].unique()
unique_positions = sorted(train_home_player_avg['POSITION'].dropna().unique())

train_home_player_avg = train_home_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Merge with the average dataset
train_home_player_avg_full = pd.merge(all_combinations_df, train_home_player_avg, on=['ID', 'POSITION'], how='left')

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_home = train_home_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_home.columns = ['_'.join(col) for col in df_player_home.columns.values]
df_player_home.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_home= pd.merge(all_ids_df, df_player_home, on='ID', how='left')

# Add 'home' suffix to column names
df_player_home = df_player_home.add_suffix('_home')
df_player_home.rename(columns={'ID_home': 'ID'}, inplace=True)

In [21]:
df_player = pd.merge(df_player_home, df_player_away, on='ID')

In [22]:
# Add 'away' suffix to column names of train_away_team
train_away_team_renamed = train_away_team_bis.add_suffix('_away')

# Add 'home' suffix to column names of train_home_team
train_home_team_renamed = train_home_team_bis.add_suffix('_home')

train = pd.concat([train_home_team_renamed, train_away_team_renamed], axis=1)
train['ID'] = range(len(train))
train = pd.merge(train, df_player, on='ID')

In [23]:
def encode_outcome(row):
    if row['HOME_WINS'] == 1:
        return 0
    elif row['DRAW'] == 1:
        return 1
    elif row['AWAY_WINS'] == 1:
        return 2

y['OUTCOME'] = y.apply(encode_outcome, axis=1)
y_bis = y[['ID', 'OUTCOME']]

In [24]:
X = train.drop('ID', axis=1)
y_outcome = y_bis['OUTCOME']

In [25]:
## Best so far
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

# Define the XGBoost classifier
xg_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, colsample_bytree=0.3, learning_rate=0.01,
                            max_depth=14, alpha=10, n_estimators=100)

# Perform K-fold cross-validation
scores = cross_val_score(xg_clf, X, y_outcome, cv=10)  # Change cv to the desired number of folds

# Print the accuracy scores for each fold
print("Accuracy scores for each fold:", scores)

# Compute the mean accuracy across all folds
mean_accuracy = scores.mean()
print("Mean accuracy:", mean_accuracy)

Accuracy scores for each fold: [0.49390739 0.4922827  0.48090983 0.4902439  0.48292683 0.49105691
 0.5        0.49918699 0.51463415 0.48373984]
Mean accuracy: 0.492888853665141


In [26]:
xg_clf.fit(X, y_outcome)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3, ...)

In [27]:
columns_to_keep = ['ID','TEAM_SHOTS_TOTAL_season_average',
                   'TEAM_SHOTS_INSIDEBOX_season_average',
                   'TEAM_SHOTS_OFF_TARGET_season_average',
                   'TEAM_SHOTS_ON_TARGET_season_average',
                   'TEAM_SHOTS_OUTSIDEBOX_season_average',
                   'TEAM_PASSES_season_average',
                   'TEAM_SUCCESSFUL_PASSES_season_average',
                   'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average',
                   'TEAM_SAVES_season_average', 'TEAM_CORNERS_season_average',
                   'TEAM_BALL_POSSESSION_season_average', 'TEAM_FOULS_season_average',
                   'TEAM_YELLOWCARDS_season_average', 'TEAM_REDCARDS_season_average',
                   'TEAM_OFFSIDES_season_average', 'TEAM_ATTACKS_season_average',
                   'TEAM_PENALTIES_season_average',
                   'TEAM_SUBSTITUTIONS_season_average',
                   'TEAM_BALL_SAFE_season_average',
                   'TEAM_DANGEROUS_ATTACKS_season_average',
                   'TEAM_INJURIES_season_average', 'TEAM_GOALS_season_average',
                   'TEAM_GAME_WON_season_average', 'TEAM_GAME_DRAW_season_average',
                   'TEAM_GAME_LOST_season_average',
                   'TEAM_SHOTS_TOTAL_5_last_match_average',
                   'TEAM_SHOTS_INSIDEBOX_5_last_match_average',
                   'TEAM_SHOTS_OFF_TARGET_5_last_match_average',
                   'TEAM_SHOTS_ON_TARGET_5_last_match_average',
                   'TEAM_SHOTS_OUTSIDEBOX_5_last_match_average',
                   'TEAM_PASSES_5_last_match_average',
                   'TEAM_SUCCESSFUL_PASSES_5_last_match_average',
                   'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average',
                   'TEAM_SAVES_5_last_match_average',
                   'TEAM_CORNERS_5_last_match_average',
                   'TEAM_BALL_POSSESSION_5_last_match_average',
                   'TEAM_FOULS_5_last_match_average',
                   'TEAM_YELLOWCARDS_5_last_match_average',
                   'TEAM_REDCARDS_5_last_match_average',
                   'TEAM_OFFSIDES_5_last_match_average',
                   'TEAM_ATTACKS_5_last_match_average',
                   'TEAM_PENALTIES_5_last_match_average',
                   'TEAM_SUBSTITUTIONS_5_last_match_average',
                   'TEAM_BALL_SAFE_5_last_match_average',
                   'TEAM_DANGEROUS_ATTACKS_5_last_match_average',
                   'TEAM_INJURIES_5_last_match_average',
                   'TEAM_GOALS_5_last_match_average',
                   'TEAM_GAME_WON_5_last_match_average',
                   'TEAM_GAME_DRAW_5_last_match_average',
                   'TEAM_GAME_LOST_5_last_match_average']

test_home_team_bis = test_home_team[columns_to_keep]
test_away_team_bis = test_away_team[columns_to_keep]

test_away_team_renamed = test_away_team.add_suffix('_away')
test_away_team_renamed.rename(columns={'ID_away': 'ID'}, inplace=True)

# Add 'home' suffix to column names of train_home_team
test_home_team_renamed = test_home_team.add_suffix('_home')
test_home_team_renamed.rename(columns={'ID_home': 'ID'}, inplace=True)
test = pd.merge(test_home_team_renamed, test_away_team_renamed, on='ID')

In [28]:
columns_to_keep = ['ID', 'POSITION',
                   'PLAYER_ACCURATE_CROSSES_5_last_match_average',
                   'PLAYER_ACCURATE_PASSES_5_last_match_average',
                   'PLAYER_ACCURATE_PASSES_PERCENTAGE_5_last_match_average',
                   'PLAYER_AERIALS_WON_5_last_match_average',
                   'PLAYER_ASSISTS_5_last_match_average',
                   'PLAYER_BIG_CHANCES_CREATED_5_last_match_average',
                   'PLAYER_BIG_CHANCES_MISSED_5_last_match_average',
                   'PLAYER_BLOCKED_SHOTS_5_last_match_average',
                   'PLAYER_CAPTAIN_5_last_match_average',
                   'PLAYER_CLEARANCES_5_last_match_average',
                   'PLAYER_CLEARANCE_OFFLINE_5_last_match_average',
                   'PLAYER_DISPOSSESSED_5_last_match_average',
                   'PLAYER_DRIBBLED_ATTEMPTS_5_last_match_average',
                   'PLAYER_DRIBBLED_PAST_5_last_match_average',
                   'PLAYER_DUELS_LOST_5_last_match_average',
                   'PLAYER_DUELS_WON_5_last_match_average',
                   'PLAYER_ERROR_LEAD_TO_GOAL_5_last_match_average',
                   'PLAYER_FOULS_5_last_match_average',
                   'PLAYER_FOULS_DRAWN_5_last_match_average',
                   'PLAYER_GOALKEEPER_GOALS_CONCEDED_5_last_match_average',
                   'PLAYER_GOALS_5_last_match_average',
                   'PLAYER_GOALS_CONCEDED_5_last_match_average',
                   'PLAYER_HIT_WOODWORK_5_last_match_average',
                   'PLAYER_INTERCEPTIONS_5_last_match_average',
                   'PLAYER_KEY_PASSES_5_last_match_average',
                   'PLAYER_MINUTES_PLAYED_5_last_match_average',
                   'PLAYER_OFFSIDES_5_last_match_average',
                   'PLAYER_OWN_GOALS_5_last_match_average',
                   'PLAYER_PASSES_5_last_match_average',
                   'PLAYER_PENALTIES_COMMITTED_5_last_match_average',
                   'PLAYER_PENALTIES_MISSES_5_last_match_average',
                   'PLAYER_PENALTIES_SAVED_5_last_match_average',
                   'PLAYER_PENALTIES_SCORED_5_last_match_average',
                   'PLAYER_PENALTIES_WON_5_last_match_average',
                   'PLAYER_RATING_5_last_match_average',
                   'PLAYER_REDCARDS_5_last_match_average',
                   'PLAYER_SAVES_5_last_match_average',
                   'PLAYER_SAVES_INSIDE_BOX_5_last_match_average',
                   'PLAYER_SHOTS_BLOCKED_5_last_match_average',
                   'PLAYER_SHOTS_ON_TARGET_5_last_match_average',
                   'PLAYER_SHOTS_TOTAL_5_last_match_average',
                   'PLAYER_STARTING_LINEUP_5_last_match_average',
                   'PLAYER_SUCCESSFUL_DRIBBLES_5_last_match_average',
                   'PLAYER_TACKLES_5_last_match_average',
                   'PLAYER_TOTAL_CROSSES_5_last_match_average',
                   'PLAYER_TOTAL_DUELS_5_last_match_average',
                   'PLAYER_YELLOWCARDS_5_last_match_average',
                   'PLAYER_PUNCHES_5_last_match_average',
                   'PLAYER_LONG_BALLS_5_last_match_average',
                   'PLAYER_LONG_BALLS_WON_5_last_match_average',
                   'PLAYER_SHOTS_OFF_TARGET_5_last_match_average', 'PLAYER_ACCURATE_CROSSES_season_average',
       'PLAYER_ACCURATE_PASSES_season_average',
       'PLAYER_ACCURATE_PASSES_PERCENTAGE_season_average',
       'PLAYER_AERIALS_WON_season_average',
       'PLAYER_ASSISTS_season_average',
       'PLAYER_BIG_CHANCES_CREATED_season_average',
       'PLAYER_BIG_CHANCES_MISSED_season_average',
       'PLAYER_BLOCKED_SHOTS_season_average',
       'PLAYER_CAPTAIN_season_average',
       'PLAYER_CLEARANCES_season_average',
       'PLAYER_CLEARANCE_OFFLINE_season_average',
       'PLAYER_DISPOSSESSED_season_average',
       'PLAYER_DRIBBLED_ATTEMPTS_season_average',
       'PLAYER_DRIBBLED_PAST_season_average',
       'PLAYER_DUELS_LOST_season_average',
       'PLAYER_DUELS_WON_season_average',
       'PLAYER_ERROR_LEAD_TO_GOAL_season_average',
       'PLAYER_FOULS_season_average', 'PLAYER_FOULS_DRAWN_season_average',
       'PLAYER_GOALKEEPER_GOALS_CONCEDED_season_average',
       'PLAYER_GOALS_season_average',
       'PLAYER_GOALS_CONCEDED_season_average',
       'PLAYER_HIT_WOODWORK_season_average',
       'PLAYER_INTERCEPTIONS_season_average',
       'PLAYER_KEY_PASSES_season_average',
       'PLAYER_MINUTES_PLAYED_season_average',
       'PLAYER_OFFSIDES_season_average',
       'PLAYER_OWN_GOALS_season_average', 'PLAYER_PASSES_season_average',
       'PLAYER_PENALTIES_COMMITTED_season_average',
       'PLAYER_PENALTIES_MISSES_season_average',
       'PLAYER_PENALTIES_SAVED_season_average',
       'PLAYER_PENALTIES_SCORED_season_average',
       'PLAYER_PENALTIES_WON_season_average',
       'PLAYER_RATING_season_average', 'PLAYER_REDCARDS_season_average',
       'PLAYER_SAVES_season_average',
       'PLAYER_SAVES_INSIDE_BOX_season_average',
       'PLAYER_SHOTS_BLOCKED_season_average',
       'PLAYER_SHOTS_ON_TARGET_season_average',
       'PLAYER_SHOTS_TOTAL_season_average',
       'PLAYER_STARTING_LINEUP_season_average',
       'PLAYER_SUCCESSFUL_DRIBBLES_season_average',
       'PLAYER_TACKLES_season_average',
       'PLAYER_TOTAL_CROSSES_season_average',
       'PLAYER_TOTAL_DUELS_season_average',
       'PLAYER_YELLOWCARDS_season_average',
       'PLAYER_PUNCHES_season_average',
       'PLAYER_LONG_BALLS_season_average',
       'PLAYER_LONG_BALLS_WON_season_average',
       'PLAYER_SHOTS_OFF_TARGET_season_average']

test_home_player_avg = test_home_player[columns_to_keep]
test_away_player_avg = test_away_player[columns_to_keep]

In [29]:
# Get all unique IDs and positions
unique_ids = test_away_player_avg['ID'].unique()
unique_positions = sorted(test_away_player_avg['POSITION'].dropna().unique())

test_away_player_avg = test_away_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Merge with the average dataset
test_away_player_avg_full = pd.merge(all_combinations_df, test_away_player_avg, on=['ID', 'POSITION'], how='left')

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_away = test_away_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_away.columns = ['_'.join(col) for col in df_player_away.columns.values]
df_player_away.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_away= pd.merge(all_ids_df, df_player_away, on='ID', how='left')

# Add 'home' suffix to column names
df_player_away = df_player_away.add_suffix('_away')
df_player_away.rename(columns={'ID_away': 'ID'}, inplace=True)

In [30]:
# Get all unique IDs and positions
unique_ids = test_home_player_avg['ID'].unique()
unique_positions = sorted(test_home_player_avg['POSITION'].dropna().unique())

test_home_player_avg = test_home_player_avg.groupby(['ID', 'POSITION']).mean().reset_index()

# Create a DataFrame with all combinations of unique IDs and positions
id_position_combinations = pd.MultiIndex.from_product([unique_ids, unique_positions], names=['ID', 'POSITION'])
all_combinations_df = pd.DataFrame(index=id_position_combinations).reset_index()

# Merge with the average dataset
test_home_player_avg_full = pd.merge(all_combinations_df, test_home_player_avg, on=['ID', 'POSITION'], how='left')

# Pivot the DataFrame to wide format, Keep one obs per ID
df_player_home = test_home_player_avg_full.pivot_table(index='ID', columns='POSITION', aggfunc='mean')
df_player_home.columns = ['_'.join(col) for col in df_player_home.columns.values]
df_player_home.reset_index(inplace=True)

# Create a DataFrame with all unique IDs
all_ids_df = pd.DataFrame({'ID': unique_ids})

# Merge the original DataFrame with all_ids_df using a left join
df_player_home= pd.merge(all_ids_df, df_player_home, on='ID', how='left')

# Add 'home' suffix to column names
df_player_home = df_player_home.add_suffix('_home')
df_player_home.rename(columns={'ID_home': 'ID'}, inplace=True)

In [31]:
df_player = pd.merge(df_player_home, df_player_away, on='ID')

In [32]:
test = pd.merge(test, df_player, on='ID')

In [33]:
test

,ID,TEAM_SHOTS_TOTAL_season_sum_home,TEAM_SHOTS_INSIDEBOX_season_sum_home,TEAM_SHOTS_OFF_TARGET_season_sum_home,TEAM_SHOTS_ON_TARGET_season_sum_home,TEAM_SHOTS_OUTSIDEBOX_season_sum_home,TEAM_PASSES_season_sum_home,TEAM_SUCCESSFUL_PASSES_season_sum_home,TEAM_SAVES_season_sum_home,TEAM_CORNERS_season_sum_home,...,PLAYER_TOTAL_DUELS_season_average_goalkeeper_away,PLAYER_TOTAL_DUELS_season_average_midfielder_away,PLAYER_YELLOWCARDS_5_last_match_average_attacker_away,PLAYER_YELLOWCARDS_5_last_match_average_defender_away,PLAYER_YELLOWCARDS_5_last_match_average_goalkeeper_away,PLAYER_YELLOWCARDS_5_last_match_average_midfielder_away,PLAYER_YELLOWCARDS_season_average_attacker_away,PLAYER_YELLOWCARDS_season_average_defender_away,PLAYER_YELLOWCARDS_season_average_goalkeeper_away,PLAYER_YELLOWCARDS_season_average_midfielder_away
0,12303,3.0,6.0,5.0,1.0,3.0,9.0,8.0,10.0,6.0,...,1.0,38.428571,33.25,10.400000,13.0,15.000000,28.000000,11.400000,4.5,22.857143
1,12304,3.0,2.0,4.0,2.0,6.0,1.0,1.0,2.0,4.0,...,1.0,33.700000,20.00,0.000000,0.0,10.000000,12.000000,4.750000,0.0,9.900000
2,12305,7.0,10.0,8.0,8.0,1.0,4.0,5.0,0.0,10.0,...,0.5,23.000000,11.00,27.500000,0.0,4.714286,9.333333,30.833333,2.5,8.285714
3,12306,5.0,7.0,5.0,9.0,5.0,7.0,6.0,3.0,5.0,...,1.5,36.625000,NaN,23.333333,0.0,17.500000,NaN,22.833333,3.0,15.500000
4,12307,5.0,4.0,6.0,3.0,5.0,0.0,0.0,2.0,2.0,...,1.0,21.857143,0.00,24.800000,0.0,10.714286,0.000000,15.600000,5.0,23.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25363,37666,5.0,5.0,5.0,5.0,7.0,6.0,6.0,4.0,6.0,...,1.5,34.500000,NaN,10.000000,0.0,6.666667,NaN,16.250000,0.0,6.166667
25364,37667,5.0,5.0,2.0,8.0,3.0,7.0,7.0,2.0,2.0,...,2.5,28.111111,NaN,7.500000,10.0,17.777778,NaN,9.625000,5.5,9.666667
25365,37668,10.0,10.0,10.0,8.0,8.0,5.0,5.0,2.0,6.0,...,1.5,35.000000,5.00,25.714286,0.0,20.000000,8.750000,16.857143,0.0,12.000000
25366,37669,4.0,3.0,5.0,3.0,7.0,7.0,7.0,0.0,4.0,...,1.0,42.500000,7.50,7.500000,0.0,0.000000,1.750000,6.500000,0.0,12.500000


In [34]:
columns_to_drop = ['ID']
X_TEST = test.drop(columns_to_drop, axis=1)

# Assuming X_test and X_TEST are pandas DataFrames
# Get the columns of each DataFrame
columns_X = set(X.columns)
columns_X_TEST = set(X_TEST.columns)

# Find columns that are in one DataFrame but not in the other
columns_missing_in_X = columns_X_TEST - columns_X
X_TEST = X_TEST.drop(columns_missing_in_X, axis=1)

In [35]:
y_pred = xg_clf.predict(X_TEST)
len(y_pred)

25368

In [36]:
test['OUTCOME'] = y_pred
Y_test = test[['ID', 'OUTCOME']]
Y_test['HOME_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 0 else 0)
Y_test['DRAW'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 1 else 0)
Y_test['AWAY_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 2 else 0)
Y_test = Y_test[['ID' , 'HOME_WINS' , 'DRAW' , 'AWAY_WINS']]

/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_26996/4064822606.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['HOME_WINS'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 0 else 0)
/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_26996/4064822606.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_test['DRAW'] = Y_test['OUTCOME'].apply(lambda x: 1 if x == 1 else 0)
/var/folders/7l/2m66nfhj1ml97dtyt2qbthc40000gn/T/ipykernel_26996/4064822606.py:5: SettingWithCopyWarning: 
A 

In [37]:
Y_test.set_index('ID', inplace=True)
Y_test

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
12303,0,0,1
12304,0,0,1
12305,1,0,0
12306,1,0,0
12307,0,0,1
...,...,...,...
37666,0,0,1
37667,1,0,0
37668,1,0,0


In [38]:
Y_test.to_csv('/Users/raphaelbarthes/Desktop/Qube Data Challenge/data/Y_test.csv')